In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Load data from sql loss_profit.db

In [46]:
# Load database .db from pipelines/data_engineer/database/loss_profit.db
from sqlalchemy import create_engine, text
from pathlib import Path

CURR_DIR = Path.cwd()
ROOT_DIR = CURR_DIR.parent.parent
DB_PATH = ROOT_DIR / 'data_engineer' / 'database' / 'loss_profit.db'

# Load data base .db and check tables
engine = create_engine(f'sqlite:///{DB_PATH}')
with engine.connect() as connection:
    result = connection.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    tables = result.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

if tables:
    for table_name in tables:
        with engine.connect() as connection:
            query = text(f"SELECT * FROM {table_name[0]}")
            df = pd.read_sql(query, connection)
            print(f"\nData from table '{table_name[0]}':")
            print(f"Database: {df.head()}")
else:
    print("No tables found in the database.")

Tables in the database:
loss_profit

Data from table 'loss_profit':
Database:      item_id  purchase_count  price  stocks   sales  stock_value profit_status
0  item_0001              34   88.0      98  2992.0       8624.0        profit
1  item_0002              37  116.0      55  4292.0       6380.0        profit
2  item_0003              49  103.0      46  5047.0       4738.0        profit
3  item_0004              21   59.0      86  1239.0       5074.0        profit
4  item_0005              49   97.0      76  4753.0       7372.0        profit


## Database as df -> Transformed database by data_engineer dir

### - **Added new column on total_revenue, stock_value, profit_status**

In [47]:
df

,item_id,purchase_count,price,stocks,sales,stock_value,profit_status
0,item_0001,34,88.0,98,2992.0,8624.0,profit
1,item_0002,37,116.0,55,4292.0,6380.0,profit
2,item_0003,49,103.0,46,5047.0,4738.0,profit
3,item_0004,21,59.0,86,1239.0,5074.0,profit
4,item_0005,49,97.0,76,4753.0,7372.0,profit
...,...,...,...,...,...,...,...
8620,item_8621,14,23.0,41,322.0,943.0,profit
8621,item_8622,4,119.0,18,476.0,2142.0,profit
8622,item_8623,42,36.0,75,1512.0,2700.0,profit
8623,item_8624,31,34.0,90,1054.0,3060.0,profit


In [48]:
df.describe()

,purchase_count,price,stocks,sales,stock_value
count,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000
mean,25.155246,83.105739,54.746783,2085.258435,4552.094493
std,14.748412,39.679490,26.085713,1686.954926,3235.795436
min,0.000000,15.000000,10.000000,0.000000,160.000000
25%,12.000000,49.000000,32.000000,710.000000,1885.000000
50%,25.000000,83.000000,54.000000,1643.000000,3741.000000
75%,38.000000,118.000000,77.000000,3108.000000,6533.000000
max,50.000000,150.000000,100.000000,7500.000000,14751.000000


## Feature engineering

In [49]:
# Bin sales into time periods
df['date'] = pd.cut(df['sales'], bins=365, labels=False)
df['date'] = pd.Timestamp('2025-01-01') + pd.to_timedelta(df['date'], unit='D')

## Data encoding

In [50]:
# Encoding profit_status into booelan values
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
encoder = LabelEncoder()
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        df_encoded[col] = encoder.fit_transform(df_encoded[col])

df_encoded.sort_values(by='date', inplace=True)
df_encoded = df_encoded.reset_index(drop=True)
df_encoded

,item_id,purchase_count,price,stocks,sales,stock_value,profit_status,date
0,3631,0,53.0,74,0.0,3922.0,0,2025-01-01
1,6900,0,145.0,81,0.0,11745.0,0,2025-01-01
2,4783,0,135.0,61,0.0,8235.0,0,2025-01-01
3,2179,0,20.0,69,0.0,1380.0,0,2025-01-01
4,4763,0,127.0,33,0.0,4191.0,0,2025-01-01
...,...,...,...,...,...,...,...,...
8620,6577,50,148.0,41,7400.0,6068.0,1,2025-12-27
8621,7381,50,149.0,55,7450.0,8195.0,1,2025-12-29
8622,7759,50,149.0,60,7450.0,8940.0,1,2025-12-29
8623,6628,50,149.0,70,7450.0,10430.0,1,2025-12-29


## Time-Based Split

In [51]:
FEATURES = [col for col in df_encoded.columns if col not in ['profit_status', 'item_id', 'date', 'sales']]
TARGET_SALES = "sales"
TARGET_PROFIT = "profit_status"

# Split BEFORE fitting scaler
train_size = int(0.55 * len(df_encoded))

X_train = df_encoded.iloc[:train_size]
X_test = df_encoded.iloc[train_size:]

# Data scaling using minmaxscaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_X = MinMaxScaler()
scaler_sales = MinMaxScaler()
scaler_profit = MinMaxScaler()

# Scaled features
X_scaled = scaler_X.fit_transform(df_encoded[FEATURES])

# Scale sales (regression)
y_sales_scaled = scaler_sales.fit_transform(
    df_encoded[[TARGET_SALES]]
)

# If profit is continuous:
y_profit_scaled = scaler_profit.fit_transform(
    df_encoded[[TARGET_PROFIT]]
)

# Build final df_scaled
df_scaled = pd.DataFrame(X_scaled, columns=FEATURES)
df_scaled["sales"] = y_sales_scaled
df_scaled["profit_status"] = y_profit_scaled

## LSTM Sequence Construction

In [53]:

# Extracting df -> LSTM Sequence Construction
def create_sequence(data, lookback=30):
    X, y_sales, y_profit = [], [], []

    for i in range(len(data) - lookback):
        X.append(data[FEATURES].iloc[i:i+lookback].values)
        y_sales.append(data[TARGET_SALES].iloc[i+lookback])
        y_profit.append(data[TARGET_PROFIT].iloc[i+lookback])

    return np.array(X), np.array(y_sales), np.array(y_profit)

lookback_period = 30
X_seq, y_sales_seq, y_profit_seq = create_sequence(df_scaled, lookback=lookback_period)

## Split into train sales and profit

In [54]:
split = int(0.55 * len(X_seq))

X_train, X_test = X_seq[:split], X_seq[split:]
y_sales_train, y_sales_test = y_sales_seq[:split], y_sales_seq[split:]
y_profit_train, y_profit_test = y_profit_seq[:split], y_profit_seq[split:]

## Build Multi-Output LSTM Model

In [55]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

timesteps = X_train.shape[1]
n_features = X_train.shape[2]

inputs = Input(shape=(timesteps, n_features))

# LSTM block (more stable size)
x = LSTM(128, return_sequences=True)(inputs)
x = Dropout(0.2)(x)

x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.2)(x)

# Shared dense layer
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)

# Output heads
sales_output = Dense(1, activation='linear', name='sales_output')(x)        # regression
profit_output = Dense(1, activation='sigmoid', name='profit_output')(x)    # binary classification

model = Model(inputs=inputs, outputs=[sales_output, profit_output])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'sales_output': 'mse',
        'profit_output': 'binary_crossentropy'
    },
    loss_weights={
        'sales_output': 1.0,
        'profit_output': 0.5
    },
    metrics={
        'sales_output': ['mae'],
        'profit_output': ['accuracy']
    }
)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 30, 4)]              0         []                            
                                                                                                  
 lstm_4 (LSTM)               (None, 30, 128)              68096     ['input_3[0][0]']             
                                                                                                  
 dropout_6 (Dropout)         (None, 30, 128)              0         ['lstm_4[0][0]']              
                                                                                                  
 lstm_5 (LSTM)               (None, 64)                   49408     ['dropout_6[0][0]']           
                                                                                            

## Model training

In [56]:
# Ensure correct shape
y_sales_train = y_sales_train.reshape(-1, 1)
y_sales_test = y_sales_test.reshape(-1, 1)

y_profit_train = y_profit_train.reshape(-1, 1)
y_profit_test = y_profit_test.reshape(-1, 1)

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=10,
    verbose=1
)

# Train model
history = model.fit(
    X_train,
    {
        "sales_output": y_sales_train,
        "profit_output": y_profit_train
    },
    validation_data=(
        X_test,
        {
            "sales_output": y_sales_test,
            "profit_output": y_profit_test
        }
    ),
    epochs=100,
    batch_size=32,
    shuffle=False,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/100


2026-03-01 23:53:41.477379: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


148/148 [==============================] - 7s 31ms/step - loss: 0.2140 - sales_output_loss: 0.1263 - profit_output_loss: 0.1754 - sales_output_mae: 0.2675 - profit_output_accuracy: 0.9300 - val_loss: 0.0835 - val_sales_output_loss: 0.0795 - val_profit_output_loss: 0.0080 - val_sales_output_mae: 0.2201 - val_profit_output_accuracy: 1.0000 - lr: 0.0010
Epoch 2/100
148/148 [==============================] - 3s 23ms/step - loss: 0.2168 - sales_output_loss: 0.1324 - profit_output_loss: 0.1690 - sales_output_mae: 0.2771 - profit_output_accuracy: 0.9710 - val_loss: 0.0904 - val_sales_output_loss: 0.0896 - val_profit_output_loss: 0.0017 - val_sales_output_mae: 0.2425 - val_profit_output_accuracy: 1.0000 - lr: 0.0010
Epoch 3/100
148/148 [==============================] - 3s 23ms/step - loss: 0.1976 - sales_output_loss: 0.0951 - profit_output_loss: 0.2050 - sales_output_mae: 0.2433 - profit_output_accuracy: 0.9710 - val_loss: 0.0907 - val_sales_output_loss: 0.0867 - val_profit_output_loss: 0.007

## Prediction with model

In [57]:
# Predict
sales_pred, profit_pred = model.predict(X_test)

# Inverse transform correctly
sales_pred_inv = scaler_sales.inverse_transform(sales_pred)
profit_pred_inv = scaler_profit.inverse_transform(profit_pred)

print("Predicted sales (first 10 rows):")
print(sales_pred_inv[:10].flatten())

print("Predicted profit (first 10 rows):")
print(profit_pred_inv[:10].flatten())

# Into dataframe
df_results = pd.DataFrame({
    "predicted_sales": sales_pred_inv.flatten(),
    "predicted_profit": profit_pred_inv.flatten()
})
df_results

121/121 [==============================] - 1s 5ms/step
Predicted sales (first 10 rows):
[1987.6935 1988.6211 1994.0914 1989.4952 1993.323  1995.4811 1988.9302
 1995.6124 1995.1439 1990.2362]
Predicted profit (first 10 rows):
[0.99185735 0.99179417 0.9917862  0.991787   0.9917915  0.9917854
 0.99177986 0.9917996  0.9917943  0.99180263]


,predicted_sales,predicted_profit
0,1987.693481,0.991857
1,1988.621094,0.991794
2,1994.091431,0.991786
3,1989.495239,0.991787
4,1993.322998,0.991791
...,...,...
3863,1932.979248,0.992446
3864,1931.661011,0.992429
3865,1934.898926,0.992430
3866,1936.332520,0.992425


## Model prediction testing

In [63]:
df_predictions = df_encoded.copy()
df_predictions[FEATURES] = scaler_X.transform(df_encoded[FEATURES])

def create_sequence(data, lookback=30):
    X = []
    for i in range(len(data) - lookback):
        X.append(data[FEATURES].iloc[i:i+lookback].values)
    return np.array(X)

lookback = 30 # loockback is period of time steps the model looks back to make a prediction (e.g., 30 days)
X_all = create_sequence(df_predictions, lookback)

sales_pred_new, profit_pred_new = model.predict(X_all)

# Inverse scale sales
sales_pred_inv_new = scaler_sales.inverse_transform(sales_pred_new)

# Convert probability → class
profit_pred_class_new = (profit_pred_new > 0.5).astype(int)

df_result = df_encoded.iloc[lookback:].copy()

df_result["predicted_sales"] = sales_pred_inv_new.flatten()
df_result["predicted_profit"] = profit_pred_class_new.flatten()

df_result.sort_values(by='item_id', ascending=False, inplace=True)
df_result['predicted_sales'] = df_result['predicted_sales'].apply(lambda x: f"{x:.2f}")
df_result['predicted_sales'] = pd.to_numeric(df_result['predicted_sales'], errors='coerce')
df_result['predicted_profit'] = df_result['predicted_profit'].map({0: 'loss', 1: 'profit'})

df_result

269/269 [==============================] - 1s 4ms/step


,item_id,purchase_count,price,stocks,sales,stock_value,profit_status,date,predicted_sales,predicted_profit
8416,8624,48,128.0,79,6144.0,10112.0,1,2025-10-27,1954.17,profit
3090,8623,31,34.0,90,1054.0,3060.0,1,2025-02-21,2007.93,profit
4095,8622,42,36.0,75,1512.0,2700.0,1,2025-03-15,1976.52,profit
1499,8621,4,119.0,18,476.0,2142.0,1,2025-01-24,2023.80,profit
1011,8620,14,23.0,41,322.0,943.0,1,2025-01-16,2053.82,profit
...,...,...,...,...,...,...,...,...,...,...
7811,4,49,97.0,76,4753.0,7372.0,1,2025-08-20,1966.73,profit
3522,3,21,59.0,86,1239.0,5074.0,1,2025-03-02,1985.32,profit
7978,2,49,103.0,46,5047.0,4738.0,1,2025-09-03,1984.78,profit
7487,1,37,116.0,55,4292.0,6380.0,1,2025-07-28,1965.96,profit


In [64]:
df_result['predicted_profit'].value_counts()

profit    8595
Name: predicted_profit, dtype: int64

In [60]:
# Create a mapping dictionary from original df
item_id_mapping = dict(zip(
    df_encoded['item_id'].astype(int),
    df['item_id'].values
))

df_result_new = df_result.copy()
# Use it to transform df_result
df_result_new['item_id'] = df_result['item_id'].astype(int).map(item_id_mapping)

print("Updated item_ids sample:")

df_result_new.sort_values(by='date', ascending=True, inplace=True)
df_result_new

Updated item_ids sample:


,item_id,purchase_count,price,stocks,sales,stock_value,profit_status,date,predicted_sales,predicted_profit
74,item_0075,0,28.0,11,0.0,308.0,0,2025-01-01,2042.13,profit
69,item_0070,0,81.0,81,0.0,6561.0,0,2025-01-01,2058.04,profit
141,item_0142,0,60.0,49,0.0,2940.0,0,2025-01-01,2062.78,profit
153,item_0154,0,37.0,99,0.0,3663.0,0,2025-01-01,2068.89,profit
94,item_0095,0,83.0,45,0.0,3735.0,0,2025-01-01,2097.29,profit
...,...,...,...,...,...,...,...,...,...,...
8620,item_8621,50,148.0,41,7400.0,6068.0,1,2025-12-27,1932.98,profit
8623,item_8624,50,149.0,70,7450.0,10430.0,1,2025-12-29,1936.33,profit
8622,item_8623,50,149.0,60,7450.0,8940.0,1,2025-12-29,1934.90,profit
8621,item_8622,50,149.0,55,7450.0,8195.0,1,2025-12-29,1931.66,profit
